# Bira-net: Bilinear attention net for diabetic retinopathy grading

In our pursuit to develop a network inspired by successful models in DR grading, we focused on leveraging and adapting the BiRA-Net architecture, which was specifically designed for this purpose. BiRA-Net combines an attention model for feature extraction and bilinear model for fine-grained classification. In our adaptation, we modified the standard BiRA-Net by replacing its ResNet component with EfficientNet for improved feature extraction.

Paper:
https://arxiv.org/abs/1905.06312

Github: 
https://github.com/ISS-Kerui/BIRA-NET-BILINEAR-ATTENTION-NET-FOR-DIABETIC-RETINOPATHY-GRADING

**Note:** We highly recommend running this notebook on a GPU. 

## 0. Initialization

In [1]:
import os

os.chdir("..")
import requests
import zipfile
from tqdm import tqdm
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from src.models.bira_net import BiraNet
from src.utils import seed_everything
from src.loading import load_data, load_test_data
from src.train import train

In [2]:
# Set seeds
seed_everything()

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



---

<a name='s1'></a>
## 1. Downloading the dataset

Fetching the dataset should take around 4-5 minutes. Unzipping takes 20s.

In [3]:
# if folder 'data/' is does not exist, download the data
if not os.path.exists("data/"):
    # Dropbox URL
    dropbox_url = "https://www.dropbox.com/scl/fi/sa14unf8s47e9ym125zgo/data.zip?rlkey=198bg0cmbmmrcjkfufy9064wm&dl=1"

    # File path where the .zip file will be saved
    file_path = "data.zip"

    response = requests.get(dropbox_url)

    if response.status_code == 200:
        with open(file_path, "wb") as file:
            file.write(response.content)
        message = "Download successful. The file has been saved as 'data.zip'."
    else:
        message = "Failed to download the file. Error code: " + str(
            response.status_code
        )

    print(message)

    # Path to the downloaded .zip file
    zip_file_path = "data.zip"

    # Directory to extract the contents of the zip file
    extraction_path = ""

    # Unzipping the file
    with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
        zip_ref.extractall(extraction_path)

    extraction_message = (
        f"The contents of the zip file have been extracted to: {extraction_path}"
    )

    print(extraction_message)

## 2. Data

Load the data with improved preprocessing. Set the batch size according to your machine, here we tried to set it as high as possible, as long as the GPU has enough memory.

In [4]:
# Load the .jpeg files in the data folder
PATH_IMAGES = "data/images_keep_ar"
PATH_LABELS = "data/labels/trainLabels.csv"
batch_size = 16
img_size = (400, 400)
num_epochs = 20
num_classes = 5

Load train and validation, 90-10 ratio. 

In [5]:
train_loader, validation_loader = load_data(
    PATH_LABELS, PATH_IMAGES, img_size, batch_size
)

Number of train samples:31964
Number of validation samples:3162


## 3. Model

Load the BiraNet model with efficientnet-b3 as backbone. The backbone is our pretrained model on diabetic retinopathy dataset.

In [6]:
# Initialize model
model = BiraNet(num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-5)
model = model.to(device)

Optional. If you want to fine tune this model, we advise you to do in in steps. Load the last checkpoint to continue with training. 

In [7]:
# MODEL = "results/models/bira_baseline.pt"
# model.load_state_dict(torch.load(MODEL))

### 3.1 Model training and evaluation

In [8]:
train(
    model,
    train_loader,
    validation_loader,
    criterion,
    optimizer,
    device,
    model_name="results/models/bira_eff_net.pt",
    num_epochs=num_epochs,
)

100%|██████████| 1998/1998 [08:19<00:00,  4.00it/s]


Epoch: 1 Loss: 0.11222338676452637
Train accuracy:  0.8451382805656363
Train kappa score:  0.7683582384254026
---------------


100%|██████████| 198/198 [00:45<00:00,  4.34it/s]


Validation accuracy:  0.8184693232131562
Validation kappa score:  0.6970448963530279
---------------
New best model saved with kappa score: 0.6970448963530279


100%|██████████| 1998/1998 [08:30<00:00,  3.92it/s]


...

Validation accuracy:  0.8159392789373814
Validation kappa score:  0.6983095301875841
---------------


100%|██████████| 1998/1998 [07:56<00:00,  4.19it/s]


Epoch: 20 Loss: 0.3165440857410431
Train accuracy:  0.859498185458641
Train kappa score:  0.8195471716253278
---------------


100%|██████████| 198/198 [00:43<00:00,  4.55it/s]

Validation accuracy:  0.8159392789373814
Validation kappa score:  0.6999211700552697
---------------


### Submission

The following code is for generating a submission file

In [9]:
# Model path
MODEL = "results/models/bira_eff_net.pt"

# Initialize model
model = BiraNet(num_classes)
model.load_state_dict(torch.load(MODEL))
model = model.to(device)

In [10]:
img_size = (400, 400)
batch_size = 16
test = load_test_data("data/test/", img_size, batch_size)

In [12]:
# Test loop
model.eval()
test_preds = []
test_names = []
with torch.no_grad():
    for images, names in tqdm(test):
        images = images.to(device)
        outputs = model(images)
        predicted = outputs.argmax(dim=1)
        test_preds.extend(predicted.cpu().numpy())
        test_names.extend(names)

100%|██████████| 3349/3349 [12:36<00:00,  4.42it/s]


In [15]:
# Make submission csv, first column "image" second columns "level"
submission = pd.DataFrame({"image": test_names, "level": test_preds})
submission.to_csv("submission.csv", index=False)